In [16]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
import os

In [3]:

FOLDER_EXECUTIONS = "../data/experiment_v05/cross_platform/executions"

In [ ]:
# scan the folders with executions 
# prepare all the possible pairs of comparisons

# get all the pairs refering to the same program_ID and from two different 
# platforms

# group the pairs per program_ID and yield the group

# so that the detector can analyze all the pairs in the same group and save a 
# single json refering to the same program_ID


In [6]:
config = load_config_and_check("../config/experiment_v05.yaml")

In [30]:
compilers = config["comparisons"][0]["compilers"]
compilers_names = [e['name'] for e in compilers]

In [63]:
all_records = []
for i_compiler in compilers_names:
    i_records = read_folder(
        folder_with_execs=FOLDER_EXECUTIONS, 
        compiler_name=i_compiler)
    all_records.extend(i_records)

df_all = pd.DataFrame.from_records(all_records)

In [68]:
def read_execution_folder(folder_with_execs, compiler_name):
    """Parse execution folder: info on the program_id and execution."""
    files = os.listdir(os.path.join(folder_with_execs, compiler_name))
    records = []
    for filename in files:
        new_record = {}
        new_record["compiler_name"] = compiler_name
        new_record["program_id"] = filename.split("_")[0]
        new_record["exec_iteration"] = filename.split("_")[1].replace(".json", "")
        new_record["filename"] = filename
        new_record["filepath"] = os.path.join(folder_with_execs, compiler_name, filename)
        records.append(new_record)
    return records

def iterate_over_program_ids(execution_folder, compilers_names):
    """Iterate over all possible program_IDs.
    
    It yields the group with all pairs of executions refering to the same
    program_ID.
    """
    all_records = []
    for i_compiler in compilers_names:
        i_records = read_execution_folder(
            folder_with_execs=execution_folder, 
            compiler_name=i_compiler)
        all_records.extend(i_records)
    
    df_all = pd.DataFrame.from_records(all_records)
    
    df_all_pairs = create_pairs(
        df_all_executions=df_all, 
        compilers_names=compilers_names)
    
    for program_id in sorted(df_all_pairs["program_id"].unique()):
        print(f"program_id: {program_id}")
        # keep only pairs of this program ID
        df_single_program_id = df_all_pairs[
            df_all_pairs["program_id"] == program_id
        ]
        # prepare the pairs of paths to the two execution results 
        # from two different platforms
        pairs_single_program_id = list(zip(
            df_single_program_id["filepath_x"],
            df_single_program_id["filepath_y"]
        ))
        yield program_id, pairs_single_program_id
    

from itertools import combinations

def create_pairs(df_all_executions, compilers_names):
    """Create all comparisons of executions from different platforms."""
    
    # get all possible pairs of platforms
    platforms_pairs = combinations(compilers_names, 2)
    
    df_pairs_all_platforms = [] 
    
    for platfrom_a, platfrom_b in platforms_pairs:
        df_a = df[df["compiler_name"] == platfrom_a]
        df_b = df[df["compiler_name"] == platfrom_b]        
        df_a_b = pd.merge(df_a, df_b, on="program_id")
        df_pairs_all_platforms.append(df_a_b)
        
    df_all_pairs = pd.concat(df_pairs_all_platforms, axis= 1)
    return df_all_pairs
    

def iterate_over_pairs_of_group(pairs):
    """It iterates over the group made of pairs of json.
    
    It yields every time two dictionary representing the exection of the two
    elements in a pair.
    """
    for path_exec_a, path_exec_b in pairs:
        with open(path_exec_a, 'r') as f:
            res_a =  json.load(f)
        with open(path_exec_b, 'r') as f:
            res_b =  json.load(f)
        plat_a = path_exec_a.split("/")[-2]
        plat_b = path_exec_b.split("/")[-2]
        yield plat_a, plat_b, res_a, res_b    
    
    


In [67]:
for program_id, group_same_program_id in iterate_over_program_ids(
    execution_folder=FOLDER_EXECUTIONS, 
    compilers_names=compilers_names):
    print("G: ", group_same_program_id)
    for plat_a, plat_b, res_a, res_b in iterate_over_pairs_of_group(group_same_program_id):
        print("res_a: ", len(res_a))
        print("res_b: ", len(res_b))
        # ran detector
        
    # save detector result for this program_ID
    

program_id: 0
G:  [('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_4.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_5.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_3.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_0.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_1.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_6.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/0_4.json', '../data/experiment_v05/cross_platform/executions/cirq/0_2.json'), ('../data/experiment_v05/cros

res_a:  690
res_b:  663
res_a:  690
res_b:  675
program_id: 10
G:  [('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_9.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_3.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_2.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_6.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_1.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_platform/executions/cirq/10_0.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/10_9.json', '../data/experiment_v05/cross_plat

res_a:  4157
res_b:  4174
res_a:  4157
res_b:  4200
res_a:  4157
res_b:  4215
res_a:  4157
res_b:  4203
res_a:  4157
res_b:  4173
res_a:  4157
res_b:  4188
res_a:  4157
res_b:  4225
res_a:  4164
res_b:  4230
res_a:  4164
res_b:  4217
res_a:  4164
res_b:  4172
res_a:  4164
res_b:  4174
res_a:  4164
res_b:  4200
res_a:  4164
res_b:  4215
res_a:  4164
res_b:  4203
res_a:  4164
res_b:  4173
res_a:  4164
res_b:  4188
res_a:  4164
res_b:  4225
res_a:  4150
res_b:  4230
res_a:  4150
res_b:  4217
res_a:  4150
res_b:  4172
res_a:  4150
res_b:  4174
res_a:  4150
res_b:  4200
res_a:  4150
res_b:  4215
res_a:  4150
res_b:  4203
res_a:  4150
res_b:  4173
res_a:  4150
res_b:  4188
res_a:  4150
res_b:  4225
res_a:  4137
res_b:  4230
res_a:  4137
res_b:  4217
res_a:  4137
res_b:  4172
res_a:  4137
res_b:  4174
res_a:  4137
res_b:  4200
res_a:  4137
res_b:  4215
res_a:  4137
res_b:  4203
res_a:  4137
res_b:  4173
res_a:  4137
res_b:  4188
res_a:  4137
res_b:  4225
res_a:  4227
res_b:  4230
res_a:  4227

res_a:  368
res_b:  365
res_a:  368
res_b:  372
res_a:  368
res_b:  368
res_a:  368
res_b:  374
res_a:  368
res_b:  369
res_a:  368
res_b:  372
res_a:  368
res_b:  370
res_a:  368
res_b:  365
res_a:  368
res_b:  362
res_a:  368
res_b:  361
res_a:  368
res_b:  365
res_a:  376
res_b:  372
res_a:  376
res_b:  368
res_a:  376
res_b:  374
res_a:  376
res_b:  369
res_a:  376
res_b:  372
res_a:  376
res_b:  370
res_a:  376
res_b:  365
res_a:  376
res_b:  362
res_a:  376
res_b:  361
res_a:  376
res_b:  365
res_a:  373
res_b:  372
res_a:  373
res_b:  368
res_a:  373
res_b:  374
res_a:  373
res_b:  369
res_a:  373
res_b:  372
res_a:  373
res_b:  370
res_a:  373
res_b:  365
res_a:  373
res_b:  362
res_a:  373
res_b:  361
res_a:  373
res_b:  365
program_id: 3
G:  [('../data/experiment_v05/cross_platform/executions/qiskit/3_8.json', '../data/experiment_v05/cross_platform/executions/cirq/3_8.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/3_8.json', '../data/experiment_v05/cross_pl

res_a:  81
res_b:  86
res_a:  81
res_b:  83
res_a:  81
res_b:  86
res_a:  86
res_b:  87
res_a:  86
res_b:  79
res_a:  86
res_b:  95
res_a:  86
res_b:  83
res_a:  86
res_b:  78
res_a:  86
res_b:  76
res_a:  86
res_b:  77
res_a:  86
res_b:  86
res_a:  86
res_b:  83
res_a:  86
res_b:  86
res_a:  87
res_b:  87
res_a:  87
res_b:  79
res_a:  87
res_b:  95
res_a:  87
res_b:  83
res_a:  87
res_b:  78
res_a:  87
res_b:  76
res_a:  87
res_b:  77
res_a:  87
res_b:  86
res_a:  87
res_b:  83
res_a:  87
res_b:  86
res_a:  81
res_b:  87
res_a:  81
res_b:  79
res_a:  81
res_b:  95
res_a:  81
res_b:  83
res_a:  81
res_b:  78
res_a:  81
res_b:  76
res_a:  81
res_b:  77
res_a:  81
res_b:  86
res_a:  81
res_b:  83
res_a:  81
res_b:  86
program_id: 6
G:  [('../data/experiment_v05/cross_platform/executions/qiskit/6_8.json', '../data/experiment_v05/cross_platform/executions/cirq/6_8.json'), ('../data/experiment_v05/cross_platform/executions/qiskit/6_8.json', '../data/experiment_v05/cross_platform/executions/

res_a:  7388
res_b:  7404
res_a:  7388
res_b:  7386
res_a:  7388
res_b:  7363
res_a:  7388
res_b:  7425
res_a:  7388
res_b:  7367
res_a:  7432
res_b:  7403
res_a:  7432
res_b:  7403
res_a:  7432
res_b:  7370
res_a:  7432
res_b:  7386
res_a:  7432
res_b:  7378
res_a:  7432
res_b:  7404
res_a:  7432
res_b:  7386
res_a:  7432
res_b:  7363
res_a:  7432
res_b:  7425
res_a:  7432
res_b:  7367
res_a:  7417
res_b:  7403
res_a:  7417
res_b:  7403
res_a:  7417
res_b:  7370
res_a:  7417
res_b:  7386
res_a:  7417
res_b:  7378
res_a:  7417
res_b:  7404
res_a:  7417
res_b:  7386
res_a:  7417
res_b:  7363
res_a:  7417
res_b:  7425
res_a:  7417
res_b:  7367
res_a:  7337
res_b:  7403
res_a:  7337
res_b:  7403
res_a:  7337
res_b:  7370
res_a:  7337
res_b:  7386
res_a:  7337
res_b:  7378
res_a:  7337
res_b:  7404
res_a:  7337
res_b:  7386
res_a:  7337
res_b:  7363
res_a:  7337
res_b:  7425
res_a:  7337
res_b:  7367
res_a:  7416
res_b:  7403
res_a:  7416
res_b:  7403
res_a:  7416
res_b:  7370
res_a:  7416

In [25]:
df_all.tail()

,compiler_name,program_id,exec_iteration,filename,filepath
235,cirq,11,7,11_7.json,../data/experiment_v05/cross_platform/executio...
236,cirq,3,6,3_6.json,../data/experiment_v05/cross_platform/executio...
237,cirq,8,7,8_7.json,../data/experiment_v05/cross_platform/executio...
238,cirq,1,4,1_4.json,../data/experiment_v05/cross_platform/executio...
239,cirq,11,8,11_8.json,../data/experiment_v05/cross_platform/executio...


In [31]:
from itertools import combinations

def iterate_over_pairs(df, compilers_names):
    # get all possible pairs of platforms
    platforms_pairs = combinations(compilers_names, 2)
    
    for platfrom_a, platfrom_b in platforms_pairs:
        df_a = df[df["compiler_name"] == platfrom_a]
        df_b = df[df["compiler_name"] == platfrom_b]        
        df = pd.merge(df_a, df_b, on="program_id")
    
    
    

In [34]:
df_res = iterate_over_pairs(df=df_all, compilers_names=compilers_names)
df_res.head()

,compiler_name_x,program_id,exec_iteration_x,filename_x,filepath_x,compiler_name_y,exec_iteration_y,filename_y,filepath_y
0,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,6,1_6.json,../data/experiment_v05/cross_platform/executio...
1,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,7,1_7.json,../data/experiment_v05/cross_platform/executio...
2,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,8,1_8.json,../data/experiment_v05/cross_platform/executio...
3,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,3,1_3.json,../data/experiment_v05/cross_platform/executio...
4,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,9,1_9.json,../data/experiment_v05/cross_platform/executio...


In [35]:
df_res.sort_values(by=["program_id", "exec_iteration_x", "exec_iteration_y"]).head()

,compiler_name_x,program_id,exec_iteration_x,filename_x,filepath_x,compiler_name_y,exec_iteration_y,filename_y,filepath_y
333,qiskit,0,0,0_0.json,../data/experiment_v05/cross_platform/executio...,cirq,0,0_0.json,../data/experiment_v05/cross_platform/executio...
334,qiskit,0,0,0_0.json,../data/experiment_v05/cross_platform/executio...,cirq,1,0_1.json,../data/experiment_v05/cross_platform/executio...
336,qiskit,0,0,0_0.json,../data/experiment_v05/cross_platform/executio...,cirq,2,0_2.json,../data/experiment_v05/cross_platform/executio...
332,qiskit,0,0,0_0.json,../data/experiment_v05/cross_platform/executio...,cirq,3,0_3.json,../data/experiment_v05/cross_platform/executio...
330,qiskit,0,0,0_0.json,../data/experiment_v05/cross_platform/executio...,cirq,4,0_4.json,../data/experiment_v05/cross_platform/executio...


In [40]:
df_res

,compiler_name_x,program_id,exec_iteration_x,filename_x,filepath_x,compiler_name_y,exec_iteration_y,filename_y,filepath_y
0,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,6,1_6.json,../data/experiment_v05/cross_platform/executio...
1,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,7,1_7.json,../data/experiment_v05/cross_platform/executio...
2,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,8,1_8.json,../data/experiment_v05/cross_platform/executio...
3,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,3,1_3.json,../data/experiment_v05/cross_platform/executio...
4,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,9,1_9.json,../data/experiment_v05/cross_platform/executio...
...,...,...,...,...,...,...,...,...,...
1195,qiskit,6,4,6_4.json,../data/experiment_v05/cross_platform/executio...,cirq,9,6_9.json,../data/experiment_v05/cross_platform/executio...
1196,qiskit,6,4,6_4.json,../data/experiment_v05/cross_platform/executio...,cirq,5,6_5.json,../data/experiment_v05/cross_platform/executio...
1197,qiskit,6,4,6_4.json,../data/experiment_v05/cross_platform/executio...,cirq,0,6_0.json,../data/experiment_v05/cross_platform/executio...
1198,qiskit,6,4,6_4.json,../data/experiment_v05/cross_platform/executio...,cirq,1,6_1.json,../data/experiment_v05/cross_platform/executio...


In [49]:
df_res[
    (df_res["program_id"] == "1") &
    (df_res["exec_iteration_x"] == "6") &
    (df_res["exec_iteration_y"] == "7")
]

,compiler_name_x,program_id,exec_iteration_x,filename_x,filepath_x,compiler_name_y,exec_iteration_y,filename_y,filepath_y
1,qiskit,1,6,1_6.json,../data/experiment_v05/cross_platform/executio...,cirq,7,1_7.json,../data/experiment_v05/cross_platform/executio...


## Read the KS results on multiple executions

In [73]:
FOLDER_DETECTOR = "../data/experiment_v05/cross_platform/predictions/ks"

all_records = []

for filename, detector_res in iterate_over(FOLDER_DETECTOR, filetype=".json", parse_json=True):
    print(f"Reading: {filename}")
    # remove the comparison
    pairs = detector_res.pop('comparisons', None)
        
    for pair in pairs:
        new_record = {**pair, **detector_res}
        all_records.append(new_record)
        
df_detector_results = pd.DataFrame.from_records(all_records)

Reading: 8
Reading: 0
Reading: 4
Reading: 5
Reading: 1
Reading: 11
Reading: 7
Reading: 2
Reading: 3
Reading: 10
Reading: 9
Reading: 6


In [74]:
df_detector_results.head()

,platform_a,platform_b,path_exec_a,path_exec_b,statistic,p_value,test,test_long_name,comparison_name,circuit_id,random_seed
0,cirq,qiskit,../data/experiment_v05/cross_platform/executio...,../data/experiment_v05/cross_platform/executio...,0.011108,0.692991,ks,Kolmogorov–Smirnov Test,cross_platform,8,None
1,cirq,qiskit,../data/experiment_v05/cross_platform/executio...,../data/experiment_v05/cross_platform/executio...,0.017212,0.176507,ks,Kolmogorov–Smirnov Test,cross_platform,8,None
2,cirq,qiskit,../data/experiment_v05/cross_platform/executio...,../data/experiment_v05/cross_platform/executio...,0.014404,0.363262,ks,Kolmogorov–Smirnov Test,cross_platform,8,None
3,cirq,qiskit,../data/experiment_v05/cross_platform/executio...,../data/experiment_v05/cross_platform/executio...,0.009277,0.872479,ks,Kolmogorov–Smirnov Test,cross_platform,8,None
4,cirq,qiskit,../data/experiment_v05/cross_platform/executio...,../data/experiment_v05/cross_platform/executio...,0.008789,0.909743,ks,Kolmogorov–Smirnov Test,cross_platform,8,None


In [77]:
df_detector_results.groupby(by="circuit_id").median()

,statistic,p_value
circuit_id,,
0,0.010193,0.788382
1,0.012573,0.536617
10,0.011780,0.620630
11,0.012329,0.562110
2,0.012085,0.587928
3,0.012024,0.594509
4,0.009705,0.835178
5,0.010620,0.744762
6,0.004211,1.000000
